In [108]:
import pandas as pd
import glob

file_paths = glob.glob("zoneRaw/Datos_SIATA/Estacion_pluviog*.csv")
df = pd.concat((pd.read_csv(file) for file in file_paths), ignore_index=True)

In [109]:
  # M     P
# 355 -> 619
# 207 -> 619
# 202 -> 311 
# 419 -> 311
# 249 -> 4
# 197 -> 25


In [110]:
df.shape

(66922335, 5)

In [111]:
codigosMedellin = [619,311,4,25] 

In [112]:
df = df[df['codigo'].isin(codigosMedellin)]

In [113]:
df.columns

Index(['codigo', 'fecha_hora', 'p1', 'p2', 'calidad'], dtype='object')

In [114]:
df.shape

(13385858, 5)

In [115]:
df = df[(df["calidad"] == 1)| (df["calidad"] == 2)]

In [116]:
df.shape

(12501290, 5)

In [117]:
columnas_fecha = ['fecha_hora']  

for columna in columnas_fecha:
    try:
        df[columna] = pd.to_datetime(df[columna], errors='coerce')
    except Exception as e:
        print(f"Error al convertir la columna {columna}: {e}")

In [122]:
fecha_minima_por_codigo = df.groupby('codigo')['fecha_hora'].min().reset_index()
fecha_minima_por_codigo = fecha_minima_por_codigo.rename(columns={'fecha_hora': 'fecha_minima'})
fecha_minima_por_codigo

,codigo,fecha_minima
0,4,2022-06-29 14:53:00
1,25,2022-06-29 14:53:00
2,311,2022-06-29 14:53:00
3,619,2022-06-29 14:53:00


In [123]:
fecha_maxima_por_codigo = df.groupby('codigo')['fecha_hora'].max().reset_index()
fecha_maxima_por_codigo = fecha_maxima_por_codigo.rename(columns={'fecha_hora': 'fecha_minima'})
fecha_maxima_por_codigo

,codigo,fecha_minima
0,4,2024-07-31 23:58:00
1,25,2024-07-31 23:58:00
2,311,2024-07-31 23:58:00
3,619,2024-07-31 23:58:00


In [120]:
fecha_minima_global = fecha_minima_por_codigo['fecha_minima'].max()
fecha_maxima_global = fecha_maxima_por_codigo['fecha_minima'].min()
print("Fecha Minimas entre las fechas maximas:", fecha_maxima_global)
print("Fecha máxima entre las fechas mínimas:", fecha_minima_global)

Fecha Minimas entre las fechas maximas: 2024-07-31 23:59:00
Fecha máxima entre las fechas mínimas: 2022-06-29 14:52:00


In [121]:
df = df[df['fecha_hora'] > fecha_minima_global]
df = df[df['fecha_hora'] < fecha_maxima_global]

In [124]:
df

,codigo,fecha_hora,p1,p2,calidad
16615004,25,2022-06-29 14:53:00,0.0,0.0,1
16615005,25,2022-06-29 14:54:00,0.0,0.0,1
16615006,25,2022-06-29 14:55:00,0.0,0.0,1
16615007,25,2022-06-29 14:56:00,0.0,0.0,1
16615008,25,2022-06-29 14:57:00,0.0,0.0,1
...,...,...,...,...,...
61444935,619,2024-07-31 23:54:00,0.0,0.0,1
61444936,619,2024-07-31 23:55:00,0.0,0.0,1
61444937,619,2024-07-31 23:56:00,0.0,0.0,1
61444938,619,2024-07-31 23:57:00,0.0,0.0,1


In [125]:
df.shape

(4266147, 5)

In [126]:
import numpy as np
df["p"] = np.where((df["p1"] ==0) & (df["p2"] ==0),df["p2"],0)
df["p"] = np.where((df["p1"] ==0) & (df["p2"] !=0),df["p2"],df["p"])
df["p"] = np.where((df["p2"] ==0) & (df["p1"] !=0),df["p1"],df["p"])

In [127]:
df.columns

Index(['codigo', 'fecha_hora', 'p1', 'p2', 'calidad', 'p'], dtype='object')

In [128]:
df = df[["codigo",	"fecha_hora",	"p"]]

In [129]:
import pandas as pd
import numpy as np
from datetime import timedelta

df = df.sort_values(by=['codigo', 'fecha_hora']).reset_index(drop=True)

df['diferencia_tiempo'] = df.groupby('codigo')['fecha_hora'].diff().dt.total_seconds() / 60

saltos_minuto = df[(df['diferencia_tiempo'] > 1) & (df['diferencia_tiempo'].notnull())]

nuevas_filas = []

for _, row in saltos_minuto.iterrows():
    fechas_faltantes = pd.date_range(
        start=row['fecha_hora'] - timedelta(minutes=row['diferencia_tiempo'] - 1),
        end=row['fecha_hora'] - timedelta(minutes=1),
        freq='1min'
    )
    nuevas_filas.extend([{
        'codigo': row['codigo'],
        'fecha_hora': fecha,
        'p': np.nan,
        'diferencia_tiempo': 1 
    } for fecha in fechas_faltantes])
    
nuevas_filas_df = pd.DataFrame(nuevas_filas)

df_completo = pd.concat([df, nuevas_filas_df]).sort_values(by=['codigo', 'fecha_hora']).reset_index(drop=True)
df_completo

,codigo,fecha_hora,p,diferencia_tiempo
0,4,2022-06-29 14:53:00,0.0,NaN
1,4,2022-06-29 14:54:00,0.0,1.0
2,4,2022-06-29 14:55:00,0.0,1.0
3,4,2022-06-29 14:56:00,0.0,1.0
4,4,2022-06-29 14:57:00,0.0,1.0
...,...,...,...,...
4397062,619,2024-07-31 23:54:00,0.0,1.0
4397063,619,2024-07-31 23:55:00,0.0,1.0
4397064,619,2024-07-31 23:56:00,0.0,1.0
4397065,619,2024-07-31 23:57:00,0.0,1.0


In [130]:
import pandas as pd

df_completo['fecha_hora'] = pd.to_datetime(df_completo['fecha_hora'])

df_completo = df_completo.sort_values(by=['codigo', 'fecha_hora'])

columns_to_interpolate = [
	'p',	
]

for col in columns_to_interpolate:
    df_completo[col] = (
        df_completo.groupby('codigo')[col] 
        .apply(lambda group: group.interpolate(method='linear')) 
        .reset_index(level=0, drop=True)  
    )

df_completo.head()

,codigo,fecha_hora,p,diferencia_tiempo
0,4,2022-06-29 14:53:00,0.0,NaN
1,4,2022-06-29 14:54:00,0.0,1.0
2,4,2022-06-29 14:55:00,0.0,1.0
3,4,2022-06-29 14:56:00,0.0,1.0
4,4,2022-06-29 14:57:00,0.0,1.0


In [131]:
df_completo = df_completo[[	'codigo',	'fecha_hora'	,'p',	'diferencia_tiempo']]
df_completo.shape

(4397067, 4)

In [132]:
df_completo['fecha_hora'] = df_completo['fecha_hora'].dt.floor('15min')

df_completo = df_completo.sort_values(by=['codigo', 'fecha_hora'])

grouped = df_completo.groupby(['codigo', 'fecha_hora']).agg(
    p_promedio=('p', 'mean'),
    p_std=('p', 'std'),
    p_sum=('p', 'sum'), 
    p_min=('p', 'min'), 
    p_max=('p', 'max')
)

grouped = grouped.reset_index()
grouped

,codigo,fecha_hora,p_promedio,p_std,p_sum,p_min,p_max
0,4,2022-06-29 14:45:00,0.0,0.0,0.0,0.0,0.0
1,4,2022-06-29 15:00:00,0.0,0.0,0.0,0.0,0.0
2,4,2022-06-29 15:15:00,0.0,0.0,0.0,0.0,0.0
3,4,2022-06-29 15:30:00,0.0,0.0,0.0,0.0,0.0
4,4,2022-06-29 15:45:00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
293135,619,2024-07-31 22:45:00,0.0,0.0,0.0,0.0,0.0
293136,619,2024-07-31 23:00:00,0.0,0.0,0.0,0.0,0.0
293137,619,2024-07-31 23:15:00,0.0,0.0,0.0,0.0,0.0
293138,619,2024-07-31 23:30:00,0.0,0.0,0.0,0.0,0.0


In [106]:
df = grouped.copy()

In [107]:
import pandas as pd


df = df.dropna()
df = df.sort_values(by=['codigo', 'fecha_hora']).reset_index(drop=True)

df['diferencia_tiempo'] = df.groupby('codigo')['fecha_hora'].diff().dt.total_seconds() / 60

saltos_minuto = df[(df['diferencia_tiempo'] != 15) & (df['diferencia_tiempo'].notnull())]

saltos_minuto['intervalos_15_min_faltantes'] = ((saltos_minuto['diferencia_tiempo'] - 15) / 15).apply(lambda x: max(0, x))

total_intervalos_15_min_faltantes = saltos_minuto['intervalos_15_min_faltantes'].sum()

porcentaje_faltante = (total_intervalos_15_min_faltantes / len(df)) * 100

# Mostrar resultados
print(f"Total de intervalos de 15 minutos faltantes: {total_intervalos_15_min_faltantes}")
print(f"Porcentaje de intervalos faltantes: {porcentaje_faltante:.2f}%")

# Mostrar registros con saltos en el tiempo
print("Registros con intervalos irregulares:")
saltos_minuto[['codigo', 'fecha_hora', 'diferencia_tiempo', 'intervalos_15_min_faltantes']]


Total de intervalos de 15 minutos faltantes: 0.0
Porcentaje de intervalos faltantes: 0.00%
Registros con intervalos irregulares:


,codigo,fecha_hora,diferencia_tiempo,intervalos_15_min_faltantes


In [103]:
saltos_minuto["intervalos_15_min_faltantes"].max()

782.0666666666667

In [133]:
import pandas as pd

grouped['fecha_hora'] = pd.to_datetime(grouped['fecha_hora'])

grouped = grouped.sort_values(by=['codigo', 'fecha_hora'])

columns_to_interpolate = [
    'p_promedio',
    'p_std',
    'p_sum',
	'p_min',	
    'p_max'
]

for col in columns_to_interpolate:
    grouped[col] = (
        grouped.groupby('codigo')[col] 
        .apply(lambda group: group.interpolate(method='linear')) 
        .reset_index(level=0, drop=True)  
    )

print(grouped.head())

   codigo          fecha_hora  p_promedio  p_std  p_sum  p_min  p_max
0       4 2022-06-29 14:45:00         0.0    0.0    0.0    0.0    0.0
1       4 2022-06-29 15:00:00         0.0    0.0    0.0    0.0    0.0
2       4 2022-06-29 15:15:00         0.0    0.0    0.0    0.0    0.0
3       4 2022-06-29 15:30:00         0.0    0.0    0.0    0.0    0.0
4       4 2022-06-29 15:45:00         0.0    0.0    0.0    0.0    0.0


In [134]:
grouped.to_csv('zoneCleaner/pluvio.csv', index=False)


In [77]:
grouped

,codigo,fecha_hora,p_promedio,p_std,p_sum,p_min,p_max
0,4,2019-12-26 15:45:00,0.0,NaN,0.0,0.0,0.0
1,4,2019-12-26 16:00:00,0.0,0.0,0.0,0.0,0.0
2,4,2019-12-26 16:15:00,0.0,0.0,0.0,0.0,0.0
3,4,2019-12-26 16:30:00,0.0,0.0,0.0,0.0,0.0
4,4,2019-12-26 16:45:00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
473409,619,2024-07-19 09:30:00,0.0,0.0,0.0,0.0,0.0
473410,619,2024-07-19 09:45:00,0.0,0.0,0.0,0.0,0.0
473411,619,2024-07-19 10:00:00,0.0,0.0,0.0,0.0,0.0
473412,619,2024-07-19 10:15:00,0.0,0.0,0.0,0.0,0.0
